In [1]:
import re
import spacy
import pandas as pd

from pykospacing import Spacing
from sklearn.feature_extraction.text import CountVectorizer

# 한국어 모델 로드
nlp = spacy.load("ko_core_news_sm")

In [2]:
spacing = Spacing()

def preprocessing(text):
    text = spacing(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 불용어 리스트 가져오기
def get_stopwords():
    stopwords = set()
    with open('../../../../../data/stopwords-ko.txt', 'r', encoding='utf-8') as file:
        for line in file:
            stopwords.add(line.strip())
    return stopwords

# 불용어 리스트 가져오기
stopwords = get_stopwords()

# 형태소 분석 및 불용어 제거 함수
def analyze_and_clean(text):
    preprocessed_text = preprocessing(text)
    doc = nlp(preprocessed_text)
    tokens = [token.text for token in doc if token.text not in stopwords]
    pos_tags = [(token.text, token.pos_) for token in doc if token.text not in stopwords]    
    return ' '.join(tokens), pos_tags

In [5]:
df = pd.read_excel('../../../../../data/filtered_news.xlsx')

# df = df[:100]
# 공백 문자열을 기본 값으로 대체
df['cleaned_feature_str'] = df['feature'].apply(lambda x: ' '.join(x.split()) if x.strip() else 'missing')

In [6]:
vectorizer = CountVectorizer(stop_words=None, lowercase=True)
X = vectorizer.fit_transform(df['cleaned_feature_str'])                           # BoW 벡터화
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())    # BoW 결과를 데이터프레임으로 변환
df_bow = pd.concat([df, bow_df], axis=1)                                          # 원래 데이터프레임과 BoW 데이터프레임을 병합